# 05_Export_Best_Model — Empaque e inferencia

- Seleccionar `best_model` desde `model/reports/best_model.json`.
- Empaquetar pipeline completo: preprocess → OCR (si aplica) → extracción → validaciones.
- Generar `model/infer.py` (listo) y `model/requirements.txt`.
- Smoke test CLI.



In [1]:
import os, json, subprocess, sys
from pathlib import Path

ROOT = Path(os.getcwd())
# Normalizar raíz para evitar notebooks/data si se ejecuta desde notebooks/
if (ROOT.name == 'notebooks') and (ROOT.parent / 'data').exists():
    ROOT = ROOT.parent
else:
    for p in [ROOT] + list(ROOT.parents):
        if (p / 'data').exists():
            ROOT = p
            break
MODEL = ROOT / 'model'
REPORTS = MODEL / 'reports'
SAMPLES = MODEL / 'samples'

# Crear requirements mínimos de inferencia
req = '''# Requisitos mínimos para inferencia
numpy
pillow
opencv-python-headless
pytesseract
pymupdf
# Opcionales para mejores resultados de OCR y modelos HF
# paddlepaddle==2.6.1
# paddleocr==2.7.3
# torch
# transformers
# scikit-learn
'''

with open(MODEL/'requirements.txt','w',encoding='utf-8') as f:
    f.write(req)

best_path = REPORTS/'best_model.json'
best = None
if best_path.exists():
    best = json.loads(best_path.read_text(encoding='utf-8'))
else:
    best = {'best_model':'Paddle+Reglas','metrics':{}}

print('Best model:', best['best_model'])

# Smoke test (si hay al menos un archivo de datos)
SAMPLES.mkdir(parents=True, exist_ok=True)
raw_imgs = list((ROOT/'data'/'raw').glob('*.png')) + list((ROOT/'data'/'raw').glob('*.jpg'))
if raw_imgs:
    test_in = str(raw_imgs[0])
    out_path = SAMPLES/'smoke_test.json'
    cmd = [sys.executable, str(MODEL/'infer.py'), '--input', test_in, '--output', str(out_path)]
    try:
        print('Ejecutando:', ' '.join(cmd))
        subprocess.run(cmd, check=False)
        print('Resultado en:', out_path)
    except Exception as e:
        print('Smoke test falló:', e)
else:
    print('No hay imágenes en data/raw para smoke test.')



Best model: LayoutLMv3
Ejecutando: c:\Users\kevin\Github\Project_IA\venv\Scripts\python.exe c:\Users\kevin\Github\Project_IA\model\infer.py --input c:\Users\kevin\Github\Project_IA\data\raw\factura_0101_p01.png --output c:\Users\kevin\Github\Project_IA\model\samples\smoke_test.json
Resultado en: c:\Users\kevin\Github\Project_IA\model\samples\smoke_test.json
